## Task 1 - SQL

### Build SparkSession:

In [2]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = (SparkSession
         .builder
         .appName('task-1')
         .getOrCreate())

### Read the json file:

In [7]:
json_path = "DataFrames_sample.json"
df_json = (spark.read.format('json')
                .option('path',json_path)
                .option('header','true')
                .option('inferSchema','true')
                .load())

### Display the schema:


In [8]:
df_json.printSchema()

root
 |-- D: double (nullable = true)
 |-- H: double (nullable = true)
 |-- HDD: string (nullable = true)
 |-- Id: long (nullable = true)
 |-- Model: string (nullable = true)
 |-- RAM: string (nullable = true)
 |-- ScreenSize: string (nullable = true)
 |-- W: double (nullable = true)
 |-- Weight: double (nullable = true)
 |-- Year: long (nullable = true)



### Get all the data when "Model" equal "MacBook Pro":




In [9]:
df_json.show(2,truncate=False)

+----+----+---------+---+-----------+----+----------+-----+------+----+
|D   |H   |HDD      |Id |Model      |RAM |ScreenSize|W    |Weight|Year|
+----+----+---------+---+-----------+----+----------+-----+------+----+
|9.48|0.61|512GB SSD|1  |MacBook Pro|16GB|15"       |13.75|4.02  |2015|
|7.74|0.52|256GB SSD|2  |MacBook    |8GB |12"       |11.04|2.03  |2016|
+----+----+---------+---+-----------+----+----------+-----+------+----+
only showing top 2 rows



In [16]:
import pyspark.sql.functions as pyfn
mackbook = df_json.select('*').where(pyfn.col('Model')=="MacBook Pro")
mackbook.show()

+----+----+---------+---+-----------+----+----------+-----+------+----+
|   D|   H|      HDD| Id|      Model| RAM|ScreenSize|    W|Weight|Year|
+----+----+---------+---+-----------+----+----------+-----+------+----+
|9.48|0.61|512GB SSD|  1|MacBook Pro|16GB|       15"|13.75|  4.02|2015|
+----+----+---------+---+-----------+----+----------+-----+------+----+



### Create TempView:

In [1]:
df_json.createOrReplaceTempView('jsonView')

NameError: name 'df_json' is not defined

### Display "RAM"column and count "RAM" column:

In [27]:
spark.sql("""SELECT RAM, COUNT(RAM) as COUNT
                FROM jsonView
                GROUP BY RAM""").show()

+----+-----+
| RAM|COUNT|
+----+-----+
|64GB|    1|
|16GB|    1|
| 8GB|    2|
+----+-----+



### Get all columns when "Year" column equal "2015"  

In [31]:
df_json.select('*').where(pyfn.col('Year')=='2015').show()

+----+----+---------+---+-----------+----+----------+-----+------+----+
|   D|   H|      HDD| Id|      Model| RAM|ScreenSize|    W|Weight|Year|
+----+----+---------+---+-----------+----+----------+-----+------+----+
|9.48|0.61|512GB SSD|  1|MacBook Pro|16GB|       15"|13.75|  4.02|2015|
+----+----+---------+---+-----------+----+----------+-----+------+----+



### Get all when "Model" start with "M":

In [36]:
spark.sql("""SELECT * 
            FROM jsonView
            WHERE Model LIKE 'M%'""").show()

+----+----+---------+---+-----------+----+----------+-----+------+----+
|   D|   H|      HDD| Id|      Model| RAM|ScreenSize|    W|Weight|Year|
+----+----+---------+---+-----------+----+----------+-----+------+----+
|9.48|0.61|512GB SSD|  1|MacBook Pro|16GB|       15"|13.75|  4.02|2015|
|7.74|0.52|256GB SSD|  2|    MacBook| 8GB|       12"|11.04|  2.03|2016|
|8.94|0.68|128GB SSD|  3|MacBook Air| 8GB|     13.3"| 12.8|  2.96|2016|
+----+----+---------+---+-----------+----+----------+-----+------+----+



### Get all data when "Model" column equal "MacBook Pro"

In [38]:
spark.sql("""SELECT * 
            FROM jsonView
            WHERE Model = 'MacBook Pro'""").show()

+----+----+---------+---+-----------+----+----------+-----+------+----+
|   D|   H|      HDD| Id|      Model| RAM|ScreenSize|    W|Weight|Year|
+----+----+---------+---+-----------+----+----------+-----+------+----+
|9.48|0.61|512GB SSD|  1|MacBook Pro|16GB|       15"|13.75|  4.02|2015|
+----+----+---------+---+-----------+----+----------+-----+------+----+



### Get all data with Multiple Conditions when "RAM" column equal "8GB" and "Model" column is "Macbook".

In [39]:
spark.sql("""SELECT * 
            FROM jsonView
            WHERE (Model = 'MacBook') AND (RAM ="8GB")""").show()

+----+----+---------+---+-------+---+----------+-----+------+----+
|   D|   H|      HDD| Id|  Model|RAM|ScreenSize|    W|Weight|Year|
+----+----+---------+---+-------+---+----------+-----+------+----+
|7.74|0.52|256GB SSD|  2|MacBook|8GB|       12"|11.04|  2.03|2016|
+----+----+---------+---+-------+---+----------+-----+------+----+



### Get all data with Multiple Conditions when "D" greater than or equal "8" and "Model" column is "iMac".

In [40]:
spark.sql("""SELECT * 
            FROM jsonView
            WHERE (Model = 'iMac') AND (D  >= 8)""").show()

+---+----+-------+---+-----+----+----------+----+------+----+
|  D|   H|    HDD| Id|Model| RAM|ScreenSize|   W|Weight|Year|
+---+----+-------+---+-----+----+----------+----+------+----+
|8.0|20.3|1TB SSD|  4| iMac|64GB|       27"|25.6|  20.8|2017|
+---+----+-------+---+-----+----+----------+----+------+----+



## Task 2


### Read "test1" dataset:

In [43]:
test1_path = "test1.csv"
df_1 = (spark.read.format('csv')
        .option('header','true')
        .option('path',test1_path)
        .option('inferSchema','true').load())

### Display Salary of the people less than or equal to 20000

In [46]:
df_1.select('*').where(pyfn.col('Salary') <= 20_000).show()

+-------+---+----------+------+
|   Name|age|Experience|Salary|
+-------+---+----------+------+
|  Sunny| 29|         4| 20000|
|   Paul| 24|         3| 20000|
| Harsha| 21|         1| 15000|
|Shubham| 23|         2| 18000|
+-------+---+----------+------+



### Display Salary of the people less than or equal to 20000 and greater than or equal 15000

In [49]:
df_1.select('*').where((pyfn.col('Salary') <= 20_000) & (pyfn.col('Salary') >= 15_000)).show()

+-------+---+----------+------+
|   Name|age|Experience|Salary|
+-------+---+----------+------+
|  Sunny| 29|         4| 20000|
|   Paul| 24|         3| 20000|
| Harsha| 21|         1| 15000|
|Shubham| 23|         2| 18000|
+-------+---+----------+------+



## Task 3 

### Read "test3" dataset:

In [69]:
test3_path = "test3.csv"
df_3 = (spark.read.format('csv')
        .option('header','true')
        .option('path',test3_path)
        .option('inferSchema','true').load())

### Display dataset

In [70]:
df_3.show()

+---------+------------+------+
|     Name| Departments|salary|
+---------+------------+------+
|    Krish|Data Science| 10000|
|    Krish|         IOT|  5000|
|   Mahesh|    Big Data|  4000|
|    Krish|    Big Data|  4000|
|   Mahesh|Data Science|  3000|
|Sudhanshu|Data Science| 20000|
|Sudhanshu|         IOT| 10000|
|Sudhanshu|    Big Data|  5000|
|    Sunny|Data Science| 10000|
|    Sunny|    Big Data|  2000|
+---------+------------+------+



### Display schema

In [71]:
df_3.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Departments: string (nullable = true)
 |-- salary: integer (nullable = true)



### Group by "Name" column and using sum function on "Name" column

In [72]:
df_3.select('Name').groupBy('Name').agg(pyfn.count('Name').alias('count')).show()

+---------+-----+
|     Name|count|
+---------+-----+
|Sudhanshu|    3|
|    Sunny|    2|
|    Krish|    3|
|   Mahesh|    2|
+---------+-----+



### Group by "Name" column and using avg function on "Name" column

In [76]:
df_3.select('Name','Salary').groupBy('Name')\
    .agg(pyfn.avg('Salary').alias('Average_Salary'))\
    .show()

+---------+------------------+
|     Name|    Average_Salary|
+---------+------------------+
|Sudhanshu|11666.666666666666|
|    Sunny|            6000.0|
|    Krish| 6333.333333333333|
|   Mahesh|            3500.0|
+---------+------------------+



### Group by "Departments" column and using sum function on "Departments" column

In [77]:
df_3.select('Departments','Salary').groupBy('Departments')\
    .agg(pyfn.sum('Salary').alias('net_Salary'))\
    .show()

+------------+----------+
| Departments|net_Salary|
+------------+----------+
|         IOT|     15000|
|    Big Data|     15000|
|Data Science|     43000|
+------------+----------+



### Group by "Departments" column and using mean function on "Departments" column:

In [79]:
df_3.select('Departments','Salary').groupBy('Departments')\
    .agg(pyfn.mean('Salary').alias('mean_Salary'))\
    .show()

+------------+-----------+
| Departments|mean_Salary|
+------------+-----------+
|         IOT|     7500.0|
|    Big Data|     3750.0|
|Data Science|    10750.0|
+------------+-----------+



Group by "Departments" column and using count function on "Departments" column:

In [80]:
df_3.select('Departments','Salary').groupBy('Departments')\
    .agg(pyfn.count('Departments').alias('count'))\
    .show()

+------------+-----+
| Departments|count|
+------------+-----+
|         IOT|    2|
|    Big Data|    4|
|Data Science|    4|
+------------+-----+



### Apply agg to using sum function get the total of salaries

In [82]:
df_3.select('Salary')\
    .agg(pyfn.sum('Salary').alias('total_Salary'))\
    .show()

+------------+
|total_Salary|
+------------+
|       73000|
+------------+



## Task 4

You've been flown to their headquarters in Ulsan, South Korea, to assist them in accurately estimating the number of crew members a ship will need.


They're currently building new ships for certain customers, and they'd like you to create a model and utilize it to estimate how many crew members the ships will require.


Metadata:
1. Measurements of ship size 
2. capacity 
3. crew 
4. age for 158 cruise ships.

It is saved in a csv file for you called "ITI_data.csv". our task is to develop a regression model that will assist in predicting the number of crew members required for future ships. The client also indicated that they have found that particular cruise lines will differ in acceptable crew counts, thus this is most likely an important factor to consider when conducting your investigation.

In [86]:
df_4_path = "ITI_data.csv"
df_4 = spark.read.format('csv').option('header','true').option('inferSchema','true').option('path',df_4_path).load()

In [98]:
trainDF, testDF = df_4.randomSplit([0.8, 0.2],seed=42)

### OneHotEncoder 


In [99]:
from pyspark.ml.feature import VectorAssembler,StringIndexer,OneHotEncoder

In [100]:
strindex = StringIndexer(inputCol='Cruise_line',outputCol='label_Cruise_line',handleInvalid='skip')
ohEncod = OneHotEncoder(inputCol='label_Cruise_line',outputCol='ohe_Cruise_line')

###Use VectorAssembler to merge all columns into one column:

In [101]:
vecassemb = VectorAssembler(inputCols=['ohe_Cruise_line','Age','Tonnage','passengers',\
                                       'length','cabins','passenger_density'],outputCol='features')

### Create a Linear Regression Model 

In [102]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(featuresCol='features',labelCol='crew')

### Creating a Pipeline

In [103]:
pipeline =Pipeline(stages = [strindex,ohEncod,vecassemb,lr])
pipeline_model = pipeline.fit(trainDF)
predDF = pipeline_model.transform(testDF)
predDF.select('features','crew','prediction').show(5)

+--------------------+----+------------------+
|            features|crew|        prediction|
+--------------------+----+------------------+
|(24,[3,18,19,20,2...| 6.0| 6.076991912972259|
|(24,[10,18,19,20,...| 5.2| 5.150815531341673|
|(24,[10,18,19,20,...| 8.5| 8.602341038488774|
|(24,[5,18,19,20,2...|6.17|5.4955417910063264|
|(24,[2,18,19,20,2...|12.0|12.889496273257606|
+--------------------+----+------------------+
only showing top 5 rows



### Model Evaluation

In [109]:
from pyspark.ml.evaluation import RegressionEvaluator
rmse_eval = RegressionEvaluator(predictionCol='prediction',labelCol='crew', metricName='rmse').evaluate(predDF)
r2_eval   = RegressionEvaluator(predictionCol='prediction',labelCol='crew', metricName='r2').evaluate(predDF)

print(f"RMSE is {rmse:.1f}")
print(f"R2 is {r2_eval}")

RMSE is 0.5
R2 is 0.9707651073148494


By Eng. Mostafa Nabieh 
If you have questions, please feel free to ask.

My Email : nabieh.mostafa@yahoo.com

My Whatsapp : +201015197566